In [18]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import time

import torch
print(torch.cuda.is_available())  # True 表示可以用 GPU

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"
)
cfg.MODEL.DEVICE = "cuda"  # 指定使用 GPU

predictor = DefaultPredictor(cfg)

print("Detectron2 模型運行裝置:", predictor.model.device)

True
Detectron2 模型運行裝置: cuda:0


In [ ]:
# analyze_pitch_from_detectron2.py
# 👉 全自動流程：影片 → Detectron2 → 三動作偵測 → 特徵分析

import os
import json
import time
import pandas as pd
from tqdm import tqdm
from pose_utils_detectron2 import load_pose_sequence
from pose_detectron2 import run_detectron2

from detect_release import detect_release
from detect_landing import detect_landing
from detect_shoulder import detect_shoulder

from landing_features import detect_landing_features
from shoulder_features import extract_shoulder_features
from release_features import extract_release_features
import warnings
warnings.filterwarnings("ignore")



# ✅ 單支影片轉特徵 json
def video_2_json(VIDEO_PATH, json_path, description, KEYPOINT_DIR):
    try:
        # 執行detectron2輸入影片 輸出KEYPOINT
        run_detectron2(video_path=VIDEO_PATH, output_dir=KEYPOINT_DIR, device='cuda')
        # 載入KEYPOINT變成pose_sequence
        pose_sequence = load_pose_sequence(KEYPOINT_DIR)
        # 偵測關鍵偵
        release_frame = detect_release(pose_sequence)
        landing_frame = detect_landing(pose_sequence, release_frame)
        shoulder_frame = detect_shoulder(pose_sequence, release_frame)
        # 組合各種特徵
        all_features = {}
        # 特徵1
        landing_features = detect_landing_features(pose_sequence, landing_frame)
        for key, value in landing_features.items():
            all_features[f'landing_features_{key}'] = value
        # 特徵2
        shoulder_features = extract_shoulder_features(pose_sequence, shoulder_frame, landing_frame)
        for key, value in shoulder_features.items():
            all_features[f'shoulder_features_{key}'] = value
        # 特徵3
        release_features = extract_release_features(pose_sequence, release_frame)
        for key, value in release_features.items():
            all_features[f'release_features_{key}'] = value
        # 給標籤
        all_features['description'] = description
        # 保存 json_path最後是檔案名不要
        #print(json_path)
        assert ('data' in json_path) and ('features_and_labels' in json_path)
        os.makedirs(os.path.dirname(json_path), exist_ok=True)
        with open(json_path, 'w') as f:
            json.dump([], f, indent=2)
            print('檔案以保存',json_path)
    except Exception as e:
        print(e,VIDEO_PATH)

ROOT_DIR='data'
# ✅ 主程式
def main():
    # 主目錄下找子目錄
    for subdir in tqdm(os.listdir(ROOT_DIR)):
        subdir_path = os.path.join(ROOT_DIR, subdir)
        # 防呆
        if not os.path.isdir(subdir_path):
            print('找不到子目錄')
            continue
        # 嘗試載入對應的 CSV，例如 Yu_Darvish_FF.csv
        # 子目錄前面通常是球員名稱
        player_name = "_".join(subdir.split("_")[:3])  # e.g., Yu_Darvish_FF
        # 通常在 主目錄/子目錄 底下有一個球員名稱csv
        csv_path = os.path.join(ROOT_DIR,subdir,f"{player_name}.csv")
        # 防呆
        if not os.path.exists(csv_path):
            print(f"⚠️ 找不到對應的 CSV：{csv_path}")
            continue
        # 找到該球員的csv
        df = pd.read_csv(csv_path)
        # 找所有該球員的影片
        for video_file in tqdm(os.listdir(subdir_path)):
            # 防呆
            if not video_file.endswith('.mp4'):
                continue
            # 影片路徑
            video_path = os.path.join(subdir_path, video_file)
            # 確定該影片的label
            row = df[df['Filename'] == video_file]
            # 防呆
            if row.empty:
                print(f"⚠️ 找不到影片對應的描述：{video_file}")
                continue
            description = row.iloc[0]['description']
            # 定義輸出資料夾 球員名稱 / features_and_labels / 影片名稱.json
            json_output_path = os.path.join(subdir_path, "features_and_labels", video_file.replace(".mp4", ".json"))
            print(json_output_path)
            # 定義KEYPOINT_DIR_path 球員名稱 / KEYPOINT_DIR_path / 影片名稱(當做一個資料夾裡面放多個jpg/npy檔案代表一個影片所有資料
            KEYPOINT_DIR = os.path.join(subdir_path, "KEYPOINT_DIR", video_file.replace(".mp4", ""))
            # 將 影片路徑(原始X) 輸出資料夾(轉換後的X和y) 標籤(y) KEYPOINT_DIR(放骨架偵測圖和關節數據)   
            video_2_json(video_path, json_output_path, description, KEYPOINT_DIR)

if __name__ == "__main__":
    main()

  0%|          | 0/201 [00:00<?, ?it/s]

data/Gerrit_Cole_CH_videos_4S/features_and_labels/pitch_0001.json
🎞️ 處理影片：data/Gerrit_Cole_CH_videos_4S/pitch_0001.mp4，總影格：238，FPS: 59.61


  0%|          | 0/238 [00:00<?, ?it/s]

👤 Frame 0: 設定目標人物 ID = 1
✅ Detectron2 處理完畢，骨架資料與圖片已儲存。



  1%|          | 2/201 [02:06<3:29:15, 63.09s/it]

...

👤 Frame 0: 設定目標人物 ID = 1
✅ Detectron2 處理完畢，骨架資料與圖片已儲存。



  4%|▍         | 8/201 [21:58<10:29:58, 195.85s/it]

0 : 84.4075927734375 219.48387145996094 688.41455078125
出手條件不滿足
1 : 100.75885009765625 229.93260192871094 678.884033203125
出手條件不滿足
2 : 84.49212646484375 230.38375854492188 698.2127685546875
出手條件不滿足
3 : 95.0086669921875 231.05152893066406 678.1528930664062
出手條件不滿足
4 : 82.49530029296875 233.13763427734375 707.23095703125
出手條件不滿足
5 : 96.93499755859375 231.3926544189453 676.2415771484375
出手條件不滿足
6 : 104.53076171875 708.727294921875 680.9244384765625
7 : 112.20343017578125 233.67149353027344 682.485595703125
出手條件不滿足
8 : 122.817626953125 195.0634765625 617.2086181640625
出手條件不滿足
9 : 108.4312744140625 224.45228576660156 687.5679321289062
出手條件不滿足
10 : 112.20751953125 711.8594360351562 711.8594360351562
出手條件不滿足
11 : 109.3160400390625 713.5601806640625 713.5601806640625
出手條件不滿足
12 : 109.45953369140625 179.4332733154297 654.2847900390625
出手條件不滿足
13 : 108.3944091796875 186.0625 715.4921264648438
出手條件不滿足
14 : 124.7603759765625 200.77023315429688 693.99267578125
出手條件不滿足
15 : 111.23968505859375 233.11

  0%|          | 0/239 [00:00<?, ?it/s]

👤 Frame 0: 設定目標人物 ID = 1
